# Algebraic expressions and the inner workings of Wick&d

## The main wick&d workflow and corresponding object classes
By now you have seen how to define operators, combine them into expression, and apply Wick's theorem to bring products of operators into normal ordered form. This process is illustrated in the following picture, together with the corresponding C++ class that encode these quantities.
So far we have encountered three main objects:
1. The `WickTheorem` class, responsible for performing Wick's theorem contractions.
1. The `OperatorExpression` class, responsible for representing a linear combination of operator products.
1. The `Expression` class, a class that represents the algebraic terms generated by contracting an operator expression.

The first two classes belong to the **diagrammatic** side of wick&d. This part of the code is represents operators with a compact representation that is convenient when applying Wick's theorem.

The `Expression` class belongs instead to the **algebraic** side of wick&d, which is responsible for representing general algebraic expressions.

<img src="fig-05-diagram.pdf">

## Hierarchy of algebraic classes

The `Expression` class represents the most general type of algebraic expression involving tensors and second quantized operators. This class is built from a series of basic classes that represent the following objects:
- `Index`: an orbital index
- `Tensor`: a tensor
- `SQOperator`: a second quantized operator
- `SymbolicTerm`: a product of tensors and second quantized operators
- `scalar_t`: a rational scalar
- `Expression`: a linear combination of symbolic terms

There is also the class `Term` to store the product of a symbolic term and a scalar. Howeverm, it is used mostly in the C++ side to pass information among classes.
The reason wick&d splits a term into a symbolic part and a scalar is because then an `Expression` can be represented with a map from a `SymbolicTerm` to a scalar. The map structure facilitates the simplification of expressions.

$$
\underbrace{
    h_{p}^{q}
}_{\mathrm{Tensor}}
\hat{a}_p^\dagger \underbrace{
    \hat{a}_q
}_{\mathrm{SQOperator}}
$$

$$
\underbrace{
\underbrace{
    h_{p}^{q} \hat{a}_p^\dagger \hat{a}_q
}_{\mathrm{Term}}
+
\underbrace{
    \frac{1}{4}
\underbrace{
    v_{pq}^{rs} \hat{a}_p^\dagger \hat{a}_q^\dagger \hat{a}_s \hat{a}_r
}_{\mathrm{SymbolicTerm}}
}_{\mathrm{Term}}
}_{\mathrm{Expression}}
$$

In [1]:
import wicked as w
from IPython.display import display, Math, Latex

def latex(expr):
    """Function to render any object that has a member latex() function"""
    display(Math(expr.latex()))

w.reset_space()
w.add_space("o", "fermion", "occupied", ['i','j','k','l','m'])
w.add_space("v", "fermion", "unoccupied", ['a','b','c','d','e','f'])    

## Orbital indices

To identify orbitals wick&d defines a class (`Index`) that stores indices efficiently.
To create an index we need to specify the space and a cardinal number associated with the index (to distinguish multiple indices that refer to the same space). Here we create the index for an occupied orbital, $o_0$, by calling the `index` function passing the string `o_0`. We can alternatively omit the underscore and just call this function with `o0`:

In [2]:
idx = w.index('o_0')
idx2 = w.index('o0')
print(idx)
print(idx2)

o0
o0


Indices have two attributes, the orbital space and position

In [3]:
idx.pos(), idx.space()

(0, 0)

When we render this to LaTeX using the (member) funtion `latex()`, wick&d uses pretty indices instead

In [4]:
print(idx)
idx.latex()

o0


'i'

In this notebook we also defined a function called `latex()` that can render any object that has a member function called `latex()`. Here is what happens if we print some indices

In [5]:
latex(w.index('o_0'))
latex(w.index('o_2'))
latex(w.index('v_0'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Second quantized operators

We can similarly construct second quantized operators for fermions and bosons using the functions `cre` and `ann`. The following is an example of creating the fermionic operators $\hat{a}_{o_0}$ and $\hat{a}^\dagger_{o_1}$. Note that the type of field (fermion/boson) is consistent with the one specified when defining the orbitals space information.

In [6]:
cre = w.cre('o0')
ann = w.ann('o1')
cre, ann

(a+(o0), a-(o1))

Creation operators are indicated with `a+`, while annihilation operators with with `a-`. A second quantized operator has function to get its properties

In [7]:
ann.field_type(), ann.type(), ann.index()

(<stat.fermion: 0>, <type.ann: 1>, o1)

## Tensors

Another basic class in wick&d is `Tensor`, used to represent tensors. Tensor creation follows an approach similar to that of creating `SQOperators`, where we have to specify the tensor indices.
Here we start by creating the tensor $T_{v_0}^{o_0}$ using the function `tensor`. This function take a label ("T"), a list of lower indices (specified as a product of space label and index cardinal number), a list of upper indices, and the tensor symmetry.

The allowed values for the tensor symmetry are:
- `w.sym.none`: No symmetry
- `w.sym.symm`: Symmetric with separate permutations of upper and lower indices
- `w.sym.anti`: Antisymmetric with separate permutations of upper and lower indices

In [8]:
t = w.tensor(label="t", lower=['v0'], upper=['o0'], symmetry=w.sym.none)
latex(t)

<IPython.core.display.Math object>

We can grab properties of tensors with the following functions

In [15]:
t.label(), t.upper(), t.lower(), t.symmetry()

('V', [o0, o1], [v0, v1], <sym.anti: 1>)

Here is a more elaborate example that builds the antisymmetric four-index tensor $V_{v_0 v_1}^{o_0 o_1}$

In [16]:
t = w.tensor("V",['v0','v1'],['o0','o1'],w.sym.anti); latex(t)

<IPython.core.display.Math object>

In [17]:
t.label(), t.upper(), t.lower(), t.symmetry()

('V', [o0, o1], [v0, v1], <sym.anti: 1>)